In [1]:
import random
import math
import numpy
import plotly
from plotly.graph_objs import Scatter, Layout


def get_linear_function_value(coefficient, bias, argument):
    return coefficient * argument + bias


def generate_sample(x_edges, y_edges, negative_part_size, positive_part_size, dividing_coefficients):
    x_min = x_edges[0]
    x_max = x_edges[1]
    y_min = y_edges[0]
    y_max = y_edges[1]
    coeff_k = dividing_coefficients[0]
    coeff_b = dividing_coefficients[1]

    x_start = x_min
    x_end = x_max
    if coeff_k > 0:
        x_start = max(x_min, (y_min - coeff_b) / coeff_k)
    if coeff_k < 0:
        x_end = min(x_max, (y_min - coeff_b) / coeff_k)
    step = (x_end - x_start) / negative_part_size
    negative_subsample = list()
    for i in range(0, negative_part_size):
        x = step * random.random() + step * i + x_start
        edge_value = min(y_max, get_linear_function_value(coeff_k, coeff_b, x))
        y = (edge_value - y_min) * random.random() + y_min
        negative_subsample.append([x, y])

    positive_subsample = list()
    x_start = x_min
    x_end = x_max
    if coeff_k > 0:
        x_end = min(x_max, (y_max - coeff_b) / coeff_k)
    if coeff_k < 0:
        x_start = max(x_min, (y_max - coeff_b) / coeff_k)
    step = (x_end - x_start) / positive_part_size
    for i in range(0, positive_part_size):
        x = step * random.random() + step * i + x_start
        edge_value = max(y_min, get_linear_function_value(coeff_k, coeff_b, x))
        y = (y_max - edge_value) * random.random() + edge_value
        positive_subsample.append([x, y])
    return [negative_subsample, positive_subsample]


def contains_wrong_item(input_sample, output_values, current_weights):
    for i in range(0, len(input_sample)):
        input_item = input_sample[i]
        if output_values[i] * (input_item[0] * current_weights[0] + input_item[1] * current_weights[1] + current_weights[2]) <= 0:
            return True
    return False


def batch_perceptron(negative_subsample, positive_subsample, start_bias):
    sample = positive_subsample + negative_subsample
    sample_size = len(sample)
    output_values = [1] * len(positive_subsample) + [-1] * len(negative_subsample)
    weights = [0, 0, start_bias]
    iteration = 0
    while True:
        if not contains_wrong_item(sample, output_values, weights):
            break
        iteration = iteration + 1
        for index in range(0, sample_size):
            input_item = sample[index]
            current_output = weights[0] * input_item[0] + weights[1] * input_item[1] + weights[2]
            if output_values[index] * current_output <= 0:
                weights[0] = weights[0] + output_values[index] * input_item[0]
                weights[1] = weights[1] + output_values[index] * input_item[1]
                weights[2] = weights[2] + output_values[index]
    print("Weigths:", weights[0],weights[1], weights[2])
    min_distance = float("inf")
    for index in range(0, sample_size):
        point = sample[index]
        distance = abs(weights[0] * point[0] + weights[1] * point[1] + weights[2]) / math.sqrt(weights[0] **2 + weights[1] **2)
        if distance < min_distance:
            min_distance = distance
    print("Iteration count ", iteration)
    print("Minimal distance", min_distance)
    return [weights, min_distance, iteration]


def display_data(x_edge_values, y_edge_values, sample, dividing_coefficients, weights):
    positive_points_x = []
    positive_points_y = []
    for point in sample[1]:
        positive_points_x.append(point[0])
        positive_points_y.append(point[1])
    positive_trace = Scatter(
        x=positive_points_x,
        y=positive_points_y,
        mode='markers',
        name = 'Positive points'
    )

    negative_points_x = []
    negative_points_y = []
    for point in sample[0]:
        negative_points_x.append(point[0])
        negative_points_y.append(point[1])
    negative_trace = Scatter(
        x=negative_points_x,
        y=negative_points_y,
        mode='markers',
        name = 'Negative points'
    )

    args = []
    desired_values = []
    result_coefficients = [-weight_values[0] / weight_values[1], -weight_values[2] / weight_values[1]]
    result_values = []
    for x in numpy.arange(x_edge_values[0], x_edge_values[1], 0.1):
        args.append(x)
        desired_values.append(get_linear_function_value(dividing_coefficients[0], dividing_coefficients[1], x))
        result_values.append(get_linear_function_value(result_coefficients[0], result_coefficients[1], x))
    desired_trace = Scatter(
        x=args,
        y=desired_values,
        mode='lines',
        name = 'Desired function'

    )
    result_trace = Scatter(
        x=args,
        y=result_values,
        mode='lines',
        name = 'Output function'
    )
    data = [desired_trace, result_trace, positive_trace, negative_trace]
    layout = Layout(title='', xaxis=dict(range=[x_edge_values[0], x_edge_values[1]]), yaxis=dict(range=[y_edge_values[0], y_edge_values[1]]))
    plotly.offline.plot(data)


def generate_dividing_coefficients(x_edge_values, y_edge_values):
    first_point = [random.uniform(x_edge_values[0], x_edge_values[1]),
                   random.uniform(y_edge_values[0], y_edge_values[1])]
    second_point = [random.uniform(x_edge_values[0], x_edge_values[1]),
                   random.uniform(y_edge_values[0], y_edge_values[1])]
    coefficient = (second_point[1] - first_point[1]) / (second_point[1] - second_point[0])
    bias = first_point[1] - coefficient * first_point[0]
    return [coefficient, bias]


x_edge_values = [-10, 1]
y_edge_values = [-10, 1]
k_size = 100
iter_values = list()
dividing_coefficients = generate_dividing_coefficients(x_edge_values, y_edge_values)
sample = generate_sample(x_edge_values, y_edge_values, k_size, k_size, dividing_coefficients)
[weight_values, min_distance, iter_count] = batch_perceptron(sample[0], sample[1], 0.0)
display_data(x_edge_values, y_edge_values, sample, dividing_coefficients, weight_values)

# k_values = [10, 20, 50, 70, 90, 100, 200, 500, 700, 800, 1000, 1500]
# generating plot
# elem = 5
# for k in k_values:
#     print("k", k)
#     total_count = 0
#     for num in range(0, elem):
#         [weight_values, min_distance, iter_count] = batch_perceptron(sample[0], sample[1], 0.0)
#         total_count = total_count + iter_count
#     total_count = total_count / elem
#     iter_values.append(total_count)
# trace = Scatter(
#         x=k_values,
#         y=iter_values
#     )
# plotly.offline.plot([trace])


Weigths: -25.600259848699928 31.732218080495834 1.0
Iteration count  13
Minimal distance 0.03461221754196304
